In [ ]:
import numpy as np
import tensorflow as tf
from scipy import misc
import glob
import imageio
import matplotlib.pyplot as plt
import random
import csv

#He K, Zhang X, Ren S, Sun J. Deep residual learning for image recognition. ...
#InProceedings of the IEEE conference on computer vision and pattern recognition 2016 (pp. 770-778).

In [ ]:
def random_files(num_files):
    x = ([[i] for i in range(num_files)])
    shuflist=random.sample(x,len(x))
    list_files=[]

    s=''
    for i in range(num_files):
        ID=str(shuflist[i][0])
        while len(ID)<5:
            ID='0'+ID
        list_files.append(ID)
        
        
    return list_files

In [ ]:
def random_translate(input_block,y_dir,x_dir):
    imsize=input_block.shape
    temp=np.empty((imsize[0],imsize[1],imsize[2]))
    y_abs=np.absolute(y_dir)
    x_abs=np.absolute(x_dir)
    
    #Vertical translation
    if y_dir != 0: 
        height_pad=np.zeros((y_abs,imsize[1],imsize[2]))
        if y_dir>0: #Shift up --> attach zeros to bottom
            temp=np.concatenate((input_block[y_abs:,:,:],height_pad),axis=0)
        else: #Shift down --> attach zeros to top
            temp=np.concatenate((height_pad,input_block[:(-1*y_abs),:,:]),axis=0)
    else: 
        temp=input_block
        
    #Horizontal translation
    if x_dir!=0:
        width_pad=np.zeros((imsize[0],x_abs,imsize[2]))
        if x_dir>0:
            temp2=np.concatenate((width_pad,temp[:,:(-1*x_abs),:]),axis=1)
        else: 
            temp2=np.concatenate((temp[:,x_abs:,:],width_pad),axis=1)
    else:
        temp2=temp
        
    return temp2

In [ ]:
#Get number of files in training set and also median frequency class weights for cross entropy loss
def get_num_files(path_ground):  
    s=''
    classtrue=[]
    f = open(path_ground, 'r')
    reader = csv.reader(f)
    for row in reader:
        if row[0]=='0':
            sublist=(1,0)
        else:
            sublist=(0,1)
    
        classtrue.append(sublist)
    f.close()
    ground_vec=classtrue

    return len(ground_vec), ground_vec

In [ ]:
# Layer wrappers
def conv_layer(inputs, channels_in, channels_out,is_training,pad_val='SAME',stvs=0.01,filter_size=1,strides=1,scopename="conv"):
    with tf.name_scope(scopename):
        s=''; weightname=(scopename,'_weights'); biasname=(scopename,'_bias')
        
        w=tf.Variable(tf.random_normal([filter_size, filter_size, channels_in, channels_out],stddev=stvs),name=s.join(weightname))
        tf.add_to_collection(tf.GraphKeys.REGULARIZATION_LOSSES, w)
        b=tf.Variable(tf.random_normal([channels_out],stddev=stvs),name=s.join(biasname))
        tf.summary.histogram(("Weight" + scopename),w)
        tf.summary.histogram(("Bias" + scopename),b)
        
        x = tf.nn.conv2d(inputs, w, strides=[1, strides, strides, 1], padding=pad_val)
        x = tf.nn.bias_add(x, b)
        #x=batch_norm_wrapper(x, is_training, decay = 0.99)
        #return tf.nn.relu(x)
        return x
        
def BNR(inputs,is_training,scopename="BNR"):
    # Batch norm + ReLu
    with tf.name_scope(scopename):
        #x=batch_norm_wrapper(inputs, is_training, decay = 0.99)
        x=tf.contrib.layers.layer_norm(inputs)
        return tf.nn.relu(x)
    
def shortcut_function(inputs,channels_in,num_filter_out,stvs=0.01,strides=1,scopename="shortcut"):
    with tf.name_scope(scopename):
        s=''; weightname=(scopename,'_weights')
        filter_size=1
        w=tf.Variable(tf.random_normal([filter_size, filter_size, channels_in, num_filter_out],stddev=stvs),name=s.join(weightname))
       
        #Use (padding = valid) because if we want to downsample with a stride of 2, the dimensions will adjust
        shortcut = tf.nn.conv2d(inputs, w, strides=[1, strides, strides, 1], padding='VALID')
        
        return shortcut
    
def maxpool2d(x, k=2, stride=2,scopename="pool"):
    with tf.name_scope(scopename):
        # MaxPool2D wrapper
        return tf.nn.max_pool(x, ksize=[1, k, k, 1], strides=[1, stride, stride, 1],padding='VALID')
       
# Batch normalization wrapper to distinguish between training and testing
# REF: https://r2rt.com/implementing-batch-normalization-in-tensorflow.html
def batch_norm_wrapper(inputs, is_training, decay = 0.99):
    
    epsilon = 1e-3
    scale = tf.Variable(tf.ones([inputs.get_shape()[-1]]))
    beta = tf.Variable(tf.zeros([inputs.get_shape()[-1]]))
    #pop_mean = tf.Variable(tf.zeros([inputs.get_shape()[-3], inputs.get_shape()[-2] ,inputs.get_shape()[-1]]), trainable=False)
    #pop_var = tf.Variable(tf.ones([inputs.get_shape()[-3], inputs.get_shape()[-2] ,inputs.get_shape()[-1]]), trainable=False)
    
    pop_mean = tf.Variable(tf.zeros([inputs.get_shape()[-1]]), trainable=False)
    pop_var = tf.Variable(tf.ones([inputs.get_shape()[-1]]), trainable=False)
  
    if is_training:
        batch_mean, batch_var = tf.nn.moments(inputs,[0,1,2])
        train_mean = tf.assign(pop_mean,
                               pop_mean * decay + batch_mean * (1 - decay))
        train_var = tf.assign(pop_var,
                              pop_var * decay + batch_var * (1 - decay))
        with tf.control_dependencies([train_mean, train_var]):
            return tf.nn.batch_normalization(inputs,
                batch_mean, batch_var, beta, scale, epsilon)
    else:
        return tf.nn.batch_normalization(inputs,
            pop_mean, pop_var, beta, scale, epsilon)
    
def resblock(inputs,channels_in,num_filter_start,num_res_layers,is_training):
    
    #The first reslayer of a block involves downsampling -- downsample the first conv layer and also shortcut projection
    inputs=reslayer(inputs,channels_in,num_filter_start,1,is_training,stride=2)
    channels_in=num_filter_start*4
    
    for i in range(1,num_res_layers):
        inputs=reslayer(inputs,channels_in,num_filter_start,0,is_training)
        
    return inputs    

    
def reslayer(inputs,channels_in,num_filter_start,s_flag,is_training,stride=1):
    
    num_filter_out=num_filter_start*4
    
    #Short cut is either a projection when downsampling or the exact same feature map
    if s_flag==0:
        shortcut=inputs
        x=BNR(inputs,is_training)
        
    else:
        x=BNR(inputs,is_training)
        shortcut=shortcut_function(x,channels_in,num_filter_out,strides=stride)
    
    # First conv
    if stride==2:
        p_val='VALID'
    else: 
        p_val='SAME'
    x=conv_layer(x, channels_in, num_filter_start, is_training,pad_val=p_val,filter_size=1,strides=stride)
    x=BNR(x,is_training)
    
    # Second conv
    x=conv_layer(x,num_filter_start,num_filter_start,is_training,filter_size=3)
    x=BNR(x,is_training)
    
    # Third conv
    x=conv_layer(x,num_filter_start,num_filter_out,is_training,filter_size=1)
    
    # Combine with shortcut
    return x + shortcut
    
    
    

In [ ]:
def resnet(x, inputdepth, picsize, is_training):
    depth_start=64
    num_classes=2
    inputdepth=1
    stvs=0.01
    
    x = tf.reshape(x, shape=[-1, picsize, picsize, inputdepth]) 
    
    # Initial conv_block
    with tf.name_scope("Initial_Conv"):
        conv_initial = conv_layer(x, inputdepth, depth_start, is_training, filter_size=7, scopename="conv_initial")
    
    # Initial max pooling
    with tf.name_scope("Max_Pool"):
        #From 256 down to 128
        pool_initial = maxpool2d(conv_initial,scopename="pool_initial")
        
    
    # Res block 1
    with tf.name_scope("Resblock1"):
        print('resblock1')
        #From 128 down to 64
        num_filter_start=64
        channels_in=depth_start
        num_res_layers=3
        resblock1=resblock(pool_initial,channels_in,num_filter_start,num_res_layers,is_training)
    
    # Res block 2
    with tf.name_scope("Resblock2"):
        #From 64 down to 32
        print('resblock2')
        channels_in=num_filter_start*4
        num_filter_start=num_filter_start*2
        num_res_layers=4
        resblock2=resblock(resblock1,channels_in,num_filter_start,num_res_layers,is_training)
    
    # Res block 3
    with tf.name_scope("Resblock3"):
        #From 32 down to 16
        print('resblock3')
        channels_in=num_filter_start*4
        num_filter_start=num_filter_start*2
        num_res_layers=6
        resblock3=resblock(resblock2,channels_in,num_filter_start,num_res_layers,is_training)
    
    # Res block 4
    with tf.name_scope("Resblock4"):
        #From 16 down to 8
        print('resblock4')
        channels_in=num_filter_start*4
        num_filter_start=num_filter_start*2
        num_res_layers=3
        resblock4=resblock(resblock3,channels_in,num_filter_start,num_res_layers,is_training)
    
    # Average pool
    with tf.name_scope("Average_Pool"):
        avg_pool=tf.nn.pool(resblock4, [2,2], "AVG", "SAME",strides=[2,2])
        
    # Fully connected
    with tf.name_scope("Dense"):
        #Flatten for fully connected
        nodes_in=int(num_filter_start*4*(picsize/(2**6))*(picsize/(2**6)))
        flatten=tf.reshape(avg_pool,[-1,nodes_in])
        densenodes=1000
        densevars={'weights':tf.Variable(tf.random_normal([nodes_in, densenodes],stddev=stvs)), 
                'biases':tf.Variable(tf.random_normal([densenodes],stddev=stvs))}
        
        tf.add_to_collection(tf.GraphKeys.REGULARIZATION_LOSSES, densevars['weights'])
        dense=tf.add(tf.matmul(flatten,densevars['weights']),densevars['biases'])
        dense=tf.nn.relu(dense)
        
    # Compress to num_classes for prediction
    with tf.name_scope("Out_Layer"):
        outvars={'weights':tf.Variable(tf.random_normal([densenodes, num_classes],stddev=stvs)), 
                'biases':tf.Variable(tf.random_normal([num_classes],stddev=stvs))}
        
        tf.add_to_collection(tf.GraphKeys.REGULARIZATION_LOSSES, outvars['weights'])
        outlayer=tf.add(tf.matmul(dense,outvars['weights']),outvars['biases'])
        print(outlayer)
        
    return outlayer
    

In [ ]:
def build_graph(trainmode, inputdepth, num_classes, picsize):
    
    #Define placeholders
    with tf.name_scope("Input"):
        X = tf.placeholder(tf.float32, [None, picsize, picsize, inputdepth])
    with tf.name_scope("Ground_Truth"):
        Y = tf.placeholder(tf.float32, [None, num_classes])
    
    #Define flow graph
    logits = resnet(X, inputdepth, picsize, trainmode) ###<---IS TRAINING

    #Prediction function for evaluating accuracy
    with tf.name_scope("Softmax"):
        prediction = tf.nn.softmax(logits)

    #Define Loss
    with tf.name_scope("Loss"):
        loss_op=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=Y))

        regularizer = tf.contrib.layers.l2_regularizer(scale=0.0001)
        reg_variables = tf.get_collection(tf.GraphKeys.REGULARIZATION_LOSSES)
        reg_term = tf.contrib.layers.apply_regularization(regularizer, reg_variables)
        loss_op += reg_term

        tf.summary.scalar("Loss",loss_op)

    #Define optimizer
    with tf.name_scope("Optimizer"):
        #optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
        optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)   

        train_op = optimizer.minimize(loss_op)
        
    # Evaluate model
    with tf.name_scope("Accuracy"):
        correct_pred = tf.equal(tf.argmax(prediction, axis=1), tf.argmax(Y, axis=1))
        accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
        tf.summary.scalar("accuracy",accuracy)

    #Confusion matrix
    with tf.name_scope("Confusion_Matrix"):
        batch_confusion = tf.confusion_matrix(tf.reshape(tf.argmax(Y,axis=1),[-1]),tf.reshape(tf.argmax(prediction,axis=1),[-1]),num_classes=num_classes,name='batch_confusion')
        confusion_image = tf.reshape( tf.cast(batch_confusion, tf.float32),[1, num_classes, num_classes, 1])
        tf.summary.image('confusion',confusion_image)

    #Define writer for Tensorboard
    writer=tf.summary.FileWriter("/output/10")
    writer_val=tf.summary.FileWriter("/output/4")
    summ=tf.summary.merge_all()

    # Initialize the variables
    init = tf.global_variables_initializer()

    #Define saver for model saver
    saver = tf.train.Saver()
    
    
    return X, Y,  logits, prediction, loss_op, train_op, accuracy, batch_confusion, confusion_image, writer, writer_val, summ, init, saver

In [ ]:
# Training Parameters
learning_rate = 0.00001
batch_size = 64
display_step = batch_size*10
hm_epochs=30
BS=batch_size
picsize=256
inputdepth=1
num_classes=2

In [ ]:
train_mode=True
tf.reset_default_graph()
X, Y, logits, prediction, loss_op, train_op, accuracy, batch_confusion, confusion_image, writer, writer_val, summ, init, saver = build_graph(train_mode, inputdepth, num_classes, picsize)
mode=''
if mode=='j':    
    imdpath="/path/to/dir/Knee_MRI2/Femur/Train/ImdTrain/"
    pxdpath="/path/to/label/csv/Femur/Train/fem_mri_classlist_Train.csv"

    imdpath_val="/path/to/dir/Knee_MRI2/Femur/Val/ImdVal/"
    pxdpath_val="/path/to/label/csv/Femur/Val/fem_mri_classlist_Val.csv"
    
    imdpath_test="/path/to/dir/Knee_MRI2/Femur/Test/ImdTest/"
    pxdpath_test="/path/to/label/csv/Femur/Test/fem_mri_classlist_Test.csv"
        
    root = '/path/to/dir/Knee_MRI2/Femur/'
    
else:
    imdpath="/mydata/Femur/Train/ImdTrain/"
    pxdpath="/mydata/Femur/Train/fem_mri_classlist_Train.csv"

    imdpath_val="/mydata/Femur/Val/ImdVal/"
    pxdpath_val="/mydata/Femur/Val/fem_mri_classlist_Val.csv"
    
    imdpath_test="/mydata/Femur/Test/ImdTest/"
    pxdpath_test="/mydata/Femur/Test/fem_mri_classlist_Test.csv"
    
    root = '/mydata/Femur/'

    
#Class data for classification of slices is stored as either 0 or 1 (not a femur, or femur) in a .csv 
#where each row corresponds to slice's file identifier in the data store

num_files_train, ground_train = get_num_files(pxdpath)

num_files_val, ground_val = get_num_files(pxdpath_val)

num_files_test, ground_test = get_num_files(pxdpath_test)


print(len(ground_train))
print(ground_train[:10])

In [ ]:
#Send it
ct=0
with tf.Session() as sess:
    sess.run(init)
    writer.add_graph(sess.graph)
    #saver.restore(sess, '/models/model_30112') #Option to restore a model to resume training
    
    for epoch in range(hm_epochs):
        
        # #List of strings of random numbers with correct # characters
        file_list=random_files(num_files_train)

        i=0
        print("EPOCH " + "{:d}".format(epoch))
        while i+BS<num_files_train:    
            start=i; end=i+BS;
            bx=np.empty((BS,picsize,picsize,inputdepth))
            by=np.empty((BS,num_classes))
            for j in range(BS): # Import data for that mini-batch
                bx[j,:,:,0]=imageio.imread((imdpath + file_list[i+j] + 'D.png')) #Import CT slice
                bx[j,:,:,0]=bx[j,:,:,0]-np.mean(bx[j,:,:,0],axis=(0,1))
                
                #Apply translation augmentation to input block
                x_stvs=15; y_stvs=15;
                x_t=int(x_stvs*np.random.randn()); y_t=int(y_stvs*np.random.randn());
                bx[j,:,:,:]=random_translate(bx[j,:,:,:],x_t,y_t)
                
                by[j,:]=ground_train[int(file_list[i+j])]

            #After setting up batch_x and batch_y, we can run train_op
            sess.run(train_op,feed_dict={X: bx, Y: by})
            loss, acc, summary=sess.run([loss_op,accuracy,summ],feed_dict={X: bx, Y: by})
            
            #conf_im=sess.run(confusion_image,feed_dict={X: bx, Y: by})
            writer.add_summary(summary, ct)
            
            if i%display_step==0 and i>0: 
                s=''
                
                checkpointnamelist=('./model_',str(ct))
                checkpointname= s.join(checkpointnamelist)
                save_path = saver.save(sess,checkpointname)
                
                #print("Model saved in file: %s" % save_path)
                print(" Step " + str(i) + ", Minibatch Loss= " + "{:.4f}".format(loss) + ", Training Accuracy = " + "{:.3f}".format(acc)) 
                print(sess.run(batch_confusion,feed_dict={X: bx, Y: by}))
                
                #Show example image from batch with ground truth
                #temp=np.squeeze(bx[0,:,:,:])
                #plt.imshow(temp,cmap='gray')
                #plt.show()
                #print(by[0,:])
    
                
            if i%(5*display_step)==0 and i>0:    
            
                acc_vec_val=[]; ct_val=0;
                for j in range(num_files_val): #Import data for that mini-batch
                    #for j in range(100): #Import data for that mini-batch

                    #Validation step -- Load entire validation set
                    bx=np.empty((1,picsize,picsize,inputdepth))
                    by=np.empty((1,num_classes))
                    batch_ID=0;
                    #Import input data
                    ID=str(j)
                    while(len(ID))<5:
                        ID='0'+ID

                    bx[batch_ID,:,:,0]=imageio.imread((imdpath_val + ID + 'D.png')) #Import CT slice         
                    bx[batch_ID,:,:,0]=bx[batch_ID,:,:,0]-np.mean(bx[batch_ID,:,:,0],axis=(0,1))

                    by[batch_ID,:]=ground_val[int(ID)]  
                    #After setting up batch_x and batch_y, we can run train_op
                    loss, acc, summary=sess.run([loss_op,accuracy,summ],feed_dict={X: bx, Y: by})
                    BC=sess.run(batch_confusion,feed_dict={X: bx, Y: by})
                    acc_vec_val.append(acc)

                print("Validation Accuracy: " + "{:.4f}".format(np.mean(acc_vec_val)))

                
            i=i+BS
            ct=ct+BS
        
    print("We out here")

In [ ]:
ctct=0
trainstep_actual=0

acc_vec=[] #Vector to store accuracy for each image
yescount=0

with tf.Session() as sess:
    sess.run(init)
    saver.restore(sess, '/models/model_84736') #Import trained model
    writer.add_graph(sess.graph)
    
    i=0
    for j in range(num_files_test): #Import data for that mini-batch

        #Validation step -- Load entire validation set
        bx=np.empty((1,picsize,picsize,inputdepth))
        by=np.empty((1,num_classes))
        batch_ID=0;
        #Import input data
        ID=str(j)
        while(len(ID))<5:
            ID='0'+ID
            
        bx[batch_ID,:,:,0]=imageio.imread((imdpath_test + ID + 'D.png')) #Import CT slice         
        bx[batch_ID,:,:,0]=bx[batch_ID,:,:,0]-np.mean(bx[batch_ID,:,:,0],axis=(0,1))

        by[batch_ID,:]=ground_test[int(ID)]  
        #After setting up batch_x and batch_y, we can run train_op
        loss, acc, summary=sess.run([loss_op,accuracy,summ],feed_dict={X: bx, Y: by})
        if acc==0:
            temp=np.squeeze(bx[0,:,:,:])
            plt.imshow(temp,cmap='gray')
            plt.show()
            print(by)
                
        BC=sess.run(batch_confusion,feed_dict={X: bx, Y: by})
        acc_vec.append(acc)

    print("Test Accuracy: " + "{:.4f}".format(np.mean(acc_vec)))

